In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from googletrans import Translator
from pandas import Series
from pandas import DataFrame
%matplotlib inline


In [2]:
item_category_df = pd.read_csv("data/item_categories.csv")
items_df = pd.read_csv("data/items.csv")
shops_df = pd.read_csv("data/shops.csv")
sample_df = pd.read_csv("data/sample_submission.csv")
train_df = pd.read_csv("data/sales_train_v2.csv")
test_df = pd.read_csv("data/test.csv")

# date-> day, month, year

In [3]:
f = lambda x: x.split(".")
train_df['date'] =list(map(f, train_df['date']))

In [4]:
f = lambda x: x[0]
train_df['day'] =list(map(f, train_df['date']))

In [5]:
f = lambda x: x[1]
train_df['month'] =list(map(f, train_df['date']))

In [6]:
f = lambda x: x[2]
train_df['year'] =list(map(f, train_df['date']))

In [7]:
train_df = train_df.drop(['date'],axis=1)

# 이상치 제거

In [8]:
icd_value = set(train_df["item_cnt_day"])

In [9]:
more_than_thousand = train_df["item_cnt_day"].where(train_df["item_cnt_day"]>999)
more_than_thousand.dropna(how='all').head()

2326930    1000.0
2909818    2169.0
Name: item_cnt_day, dtype: float64

In [10]:

below_ten = train_df["item_cnt_day"].where(train_df["item_cnt_day"] <-8)
below_ten.dropna(how='all').head()

1057907   -22.0
1835454   -16.0
2048519    -9.0
Name: item_cnt_day, dtype: float64

In [11]:
train_new = train_df.drop([2326930,2909818,1057907,1835454,2048519])

In [12]:
train_df = train_new

In [13]:
y_data = train_df.item_cnt_day.reshape(train_df.item_cnt_day.size,1)

c:\users\soosg\appdata\local\continuum\miniconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [14]:
train_df = train_df.drop(['item_cnt_day'],axis=1)

## 번역 (러시아어 -> 영어) -> tf-idf

In [15]:
trans = Translator()
def translate(column):
    en_list=[]
    for word in column:
        en = trans.translate(word, dest='en')
        en_list.append(en.text)
    return en_list

In [16]:
shops_df["shop_name"] = translate(shops_df["shop_name"])
shops_df.head()

,shop_name,shop_id
0,"Yakutsk Ordzhonikidze, 56 francs",0
1,"Yakutsk TC ""Central"" franc",1
2,"Adygea Shopping Center ""Mega""",2
3,"Balashikha TRK ""October-Kinomir""",3
4,"Volzhsky shopping center ""Volga Mall""",4


In [17]:
en_shop_name = np.asarray(shops_df.shop_name)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), min_df=0.03, max_df = 0.95)
shop_name_model = vectorizer.fit_transform(en_shop_name).toarray()

In [19]:
shop_X =[]
for i in range(len(en_shop_name)):
    shop_X.append(shop_name_model[i])

In [20]:
shops_df["shop_name_tfidf"] = shop_X
shops_df.head()

,shop_name,shop_id,shop_name_tfidf
0,"Yakutsk Ordzhonikidze, 56 francs",0,"[0.416609635057569, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
1,"Yakutsk TC ""Central"" franc",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53929388..."
2,"Adygea Shopping Center ""Mega""",2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4538104235202..."
3,"Balashikha TRK ""October-Kinomir""",3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"Volzhsky shopping center ""Volga Mall""",4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4320054028830..."


In [21]:
for i in range(len(shops_df)):
    for j in range(len(shops_df['shop_name_tfidf'][0])):
        shops_df['shop_name_tfidf' + str(j)]= shops_df['shop_name_tfidf'][i][j]

In [22]:
shops_df = shops_df.drop(['shop_name_tfidf'],axis=1)
shops_df.head()

,shop_name,shop_id,shop_name_tfidf0,shop_name_tfidf1,shop_name_tfidf2,shop_name_tfidf3,shop_name_tfidf4,shop_name_tfidf5,shop_name_tfidf6,shop_name_tfidf7,...,shop_name_tfidf65,shop_name_tfidf66,shop_name_tfidf67,shop_name_tfidf68,shop_name_tfidf69,shop_name_tfidf70,shop_name_tfidf71,shop_name_tfidf72,shop_name_tfidf73,shop_name_tfidf74
0,"Yakutsk Ordzhonikidze, 56 francs",0,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Yakutsk TC ""Central"" franc",1,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Adygea Shopping Center ""Mega""",2,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Balashikha TRK ""October-Kinomir""",3,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Volzhsky shopping center ""Volga Mall""",4,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
item_category_df["item_category_name"] = translate(item_category_df["item_category_name"])
item_category_df.head()

,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4


In [24]:
en_item_category_name = np.asarray(item_category_df.item_category_name)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), min_df=0.03, max_df = 0.95)
item_category_name_model = vectorizer.fit_transform(en_item_category_name).toarray()

In [26]:
item_category_X =[]
for i in range(len(en_item_category_name)):
    item_category_X.append(item_category_name_model[i])

In [27]:
item_category_df["item_category_tfidf"] = item_category_X
item_category_df.head()

,item_category_name,item_category_id,item_category_tfidf
0,PC - Headsets / Headphones,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Accessories - PS2,1,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."
2,Accessories - PS3,2,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."
3,Accessories - PS4,3,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."
4,Accessories - PSP,4,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."


In [28]:
for i in range(len(item_category_df)):
    for j in range(len(item_category_df['item_category_tfidf'][0])):
        item_category_df['item_category_tfidf' + str(j)]= item_category_df['item_category_tfidf'][i][j]

In [29]:
item_category_df = item_category_df.drop(['item_category_tfidf'],axis=1)
item_category_df.head()

,item_category_name,item_category_id,item_category_tfidf0,item_category_tfidf1,item_category_tfidf2,item_category_tfidf3,item_category_tfidf4,item_category_tfidf5,item_category_tfidf6,item_category_tfidf7,...,item_category_tfidf28,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37
0,PC - Headsets / Headphones,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Accessories - PS2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Accessories - PS3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Accessories - PS4,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Accessories - PSP,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
items_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [31]:
rus_item_name = np.asarray(items_df.item_name)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), min_df=0.03, max_df = 0.95)
rus_item_name_model = vectorizer.fit_transform(rus_item_name).toarray()

In [33]:
item_X =[]
for i in range(len(rus_item_name)):
    item_X.append(rus_item_name_model[i])

In [34]:
items_df["item_tfidf"] = item_X
items_df.head()

,item_name,item_id,item_category_id,item_tfidf
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,"[0.0, 0.0, 0.0, 0.0, 0.4825704672175713, 0.0, ..."
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,***КОРОБКА (СТЕКЛО) D,4,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [35]:
for i in range(len(items_df)):
    for j in range(len(items_df['item_tfidf'][0])):
        items_df['item_tfidf' + str(j)]= items_df['item_tfidf'][i][j]

In [36]:
items_df = items_df.drop(['item_tfidf'],axis=1)
items_df.head()

,item_name,item_id,item_category_id,item_tfidf0,item_tfidf1,item_tfidf2,item_tfidf3,item_tfidf4,item_tfidf5,item_tfidf6,...,item_tfidf13,item_tfidf14,item_tfidf15,item_tfidf16,item_tfidf17,item_tfidf18,item_tfidf19,item_tfidf20,item_tfidf21,item_tfidf22
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,***КОРОБКА (СТЕКЛО) D,4,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# merge

In [37]:
df =pd.merge(train_df, shops_df, on='shop_id')
df.head()

,date_block_num,shop_id,item_id,item_price,day,month,year,shop_name,shop_name_tfidf0,shop_name_tfidf1,...,shop_name_tfidf65,shop_name_tfidf66,shop_name_tfidf67,shop_name_tfidf68,shop_name_tfidf69,shop_name_tfidf70,shop_name_tfidf71,shop_name_tfidf72,shop_name_tfidf73,shop_name_tfidf74
0,0,59,22154,999.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,59,22151,399.0,10,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,59,5603,699.0,04,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,59,5587,199.0,19,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,59,5613,5571.0,31,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
df =pd.merge(df, items_df, on='item_id')
df.head()

,date_block_num,shop_id,item_id,item_price,day,month,year,shop_name,shop_name_tfidf0,shop_name_tfidf1,...,item_tfidf13,item_tfidf14,item_tfidf15,item_tfidf16,item_tfidf17,item_tfidf18,item_tfidf19,item_tfidf20,item_tfidf21,item_tfidf22
0,0,59,22154,999.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,25,22154,999.0,02,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,25,22154,999.0,03,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,25,22154,999.0,20,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,25,22154,999.0,23,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
df =pd.merge(df, item_category_df, on='item_category_id')
df.head()

,date_block_num,shop_id,item_id,item_price,day,month,year,shop_name,shop_name_tfidf0,shop_name_tfidf1,...,item_category_tfidf28,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37
0,0,59,22154,999.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,25,22154,999.0,02,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,25,22154,999.0,03,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,25,22154,999.0,20,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,25,22154,999.0,23,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
df = df.drop(['item_name'],axis=1)
df = df.drop(['item_category_name'],axis=1)

MemoryError: 

In [ ]:
df = df.drop(['day'],axis=1)
# df = df.drop(['year'],axis=1)

In [ ]:
df = df.drop(['item_price'],axis=1)

In [ ]:
df.head()

# location feature


In [ ]:
loc_shop = df["shop_name"]
len(loc_shop)

In [ ]:
loc = []
for i in loc_shop:
    splited_loc = i.split()
    loc.append(splited_loc[0])

In [ ]:
df['location'] = loc

In [ ]:
west= ['Adygea','Balashikha', 'Chekhov','Kaluga','Kazan', 'Krasnoyarsk' ,'Kursk', 'Moscow' ,'Yaroslavl',
        'Mytischi', 'N.Novgorod', 'Nizhny','RostovNaDon',  'RostovNaDonu', 'Samara' ,'SPb' ,'Sergiev', 
        'Surgut', 'Tyumen', 'Ufa', 'Vologda', 'Volzhsky', 'Voronezh' , 'Zhukovsky','Colosseum', 'Khimki']
east =  ['Yakutsk']
south =  ['Novosibirsk','Omsk', 'Tomsk']

In [ ]:
s1 = Series(data = 'west', index= west)
west = s1.to_dict()

s2 = Series(data = 'east'
   , index= east)
east = s2.to_dict()

s3 = Series(data = 'south'
   , index= south)
south = s3.to_dict()

In [ ]:
df["location"].replace(to_replace=west.keys(), value=west.values(), inplace=True)
df["location"].replace(to_replace=east.keys(), value=east.values(), inplace=True)
df["location"].replace(to_replace=south.keys(), value=south.values(), inplace=True)
df.head()

In [ ]:
dummy_loc_df =pd.get_dummies(df['location'])
dummy_loc_df.head()

In [ ]:
df = pd.concat([df, dummy_loc_df],axis=1)
df = df.drop(['location'],axis=1)
df.head()

In [ ]:
# df = df.drop(['item_name'],axis=1)
# df = df.drop(['item_category_name'],axis=1)
df = df.drop(['shop_name'],axis=1)

In [ ]:
df.head()

# 가격변동

In [ ]:
train_df.head()

In [ ]:
pro_min_df = train_df.groupby(["shop_id","item_id","month","year","date_block_num"]).agg({"item_price":"min"}).unstack().fillna(0)
pro_min_df.head()

In [ ]:
pro_max_df = train_df.groupby(["shop_id","item_id","month","year","date_block_num"]).agg({"item_price":"max"}).unstack().fillna(0)
pro_max_df.head()

In [ ]:
pro_min_df.equals(pro_max_df) # 가격의 변동이 있다

In [ ]:
pro_df = pro_min_df.where((pro_min_df == pro_max_df), other = np.nan)  # 최대와 최소 값 비교, 다르면 nan

In [ ]:
null_columns=pro_df.columns[pro_df.isnull().any()]
sum(pro_df[null_columns].isnull().sum()) #총 nan값

In [ ]:
diffenent_price = (pro_df[pro_df.isnull().any(axis=1)][null_columns]) 

In [ ]:
diffenent_price["price_fluctuation"] =np.ones(len(diffenent_price))

In [ ]:
diffenent_price = diffenent_price.drop("item_price",axis =1 )

In [ ]:
diffenent_price = diffenent_price.reset_index()
diffenent_price.head()

In [ ]:
diffenent_price.head()
len(diffenent_price)

In [ ]:
df.head()
len(df)

In [ ]:
df.head()

In [ ]:
price_fluctuation_df =pd.merge(df, diffenent_price, on=['shop_id','item_id','month','year'], how='outer').fillna(0)
price_fluctuation_df.head()

In [ ]:
price_fluctuation_df.head()
len(price_fluctuation_df)

In [ ]:
price_fluctuation_df = price_fluctuation_df.drop(price_fluctuation_df.columns.values[-2],axis =1 )

In [ ]:
price_fluctuation_df.head()

In [ ]:
price_fluctuation_df.columns.values[-1] = 'price_fluctuation'

In [ ]:
price_fluctuation_df.shape

In [ ]:
train_df = price_fluctuation_df
train_df.head()

In [ ]:
train_df['ID'] = train_df.index
train_df.head()

# test

In [ ]:
test_df = pd.read_csv("data/test.csv")

In [ ]:
test_df.head()

In [ ]:
test_df =pd.merge(test_df, items_df, on='item_id')
# test_df.head()

In [ ]:
test_df =pd.merge(test_df, shops_df, on='shop_id')
# test_df.head()

In [ ]:
test_df =pd.merge(test_df, item_category_df, on='item_category_id')
# test_df.head()

In [ ]:
loc_shop = test_df["shop_name"]
len(loc_shop)

In [ ]:
loc = []
for i in loc_shop:
    splited_loc = i.split()
    loc.append(splited_loc[0])

In [ ]:
test_df['location'] = loc

In [ ]:
west= ['Adygea','Balashikha', 'Chekhov','Kaluga','Kazan', 'Krasnoyarsk' ,'Kursk', 'Moscow' ,'Yaroslavl',
        'Mytischi', 'N.Novgorod', 'Nizhny','RostovNaDon',  'RostovNaDonu', 'Samara' ,'SPb' ,'Sergiev', 
        'Surgut', 'Tyumen', 'Ufa', 'Vologda', 'Volzhsky', 'Voronezh' , 'Zhukovsky','Colosseum', 'Khimki']
east =  ['Yakutsk']
south =  ['Novosibirsk','Omsk', 'Tomsk']

In [ ]:
s1 = Series(data = 'west', index= west)
west = s1.to_dict()

s2 = Series(data = 'east'
   , index= east)
east = s2.to_dict()

s3 = Series(data = 'south'
   , index= south)
south = s3.to_dict()

In [ ]:
test_df["location"].replace(to_replace=west.keys(), value=west.values(), inplace=True)
test_df["location"].replace(to_replace=east.keys(), value=east.values(), inplace=True)
test_df["location"].replace(to_replace=south.keys(), value=south.values(), inplace=True)
test_df.head()

In [ ]:
dummy_loc_df =pd.get_dummies(test_df['location'])
dummy_loc_df.head()

In [ ]:
test_df = pd.concat([test_df, dummy_loc_df],axis=1)
test_df = test_df.drop(['location'],axis=1)
test_df.head()

In [ ]:
test_df = test_df.drop(['item_name'],axis=1)
test_df = test_df.drop(['item_category_name'],axis=1)
test_df = test_df.drop(['item_category_id'],axis=1)
test_df = test_df.drop(['shop_name'],axis=1)
test_df.head()

In [ ]:
test_df['month'] =11
test_df['year'] = 2015
test_df['date_block_num']= 34
test_df["price_fluctuation"] = 0
test_df["Outbound"] = 0
test_df = test_df.sort_values("ID")
test_df.head()

In [ ]:
test_df.columns

In [ ]:
train_df.columns

# Evaluation

In [ ]:
y_data.shape

In [ ]:
test_data = test_df.values

In [ ]:
test_data.shape

In [ ]:
x_data = train_df.values

In [ ]:
x_data.shape

In [ ]:
from sklearn.linear_model import Lasso,LinearRegression
lr_simple = LinearRegression().fit(x_data, y_data)
lr_lasso = Lasso().fit(x_data, y_data)

In [ ]:
y_hat_simple = lr_simple.predict(test_data)

In [ ]:
simple_lr_result = pd.DataFrame(y_hat_simple, columns=["item_cnt_month"])
simple_lr_result.index.name = 'ID'
simple_lr_result.to_csv("simple_lr_result.csv", sep=",")
simple_lr_result.head()

In [ ]:
y_hat_lasso = lr_lasso.predict(test_data)

In [ ]:
lasso_lr_result = pd.DataFrame(y_hat_lasso, columns=["item_cnt_month"])
lasso_lr_result.index.name = 'ID'
lasso_lr_result.to_csv("lasso_lr_result.csv", sep=",")
lasso_lr_result.head()